# 导入数据

In [ ]:
"""author Binny"""
# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')
import sys
# 导入处理数据包
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\train.csv')
test_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\test.csv')
full = train_data.append(test_data, ignore_index=True)
print('训练集大小：', train_data.shape, '测试集大小：', test_data.shape)
print('合并后的数据集大小：', full.shape)
print('合并后的数据集前5行：\n', full.head())

In [ ]:
'''
describe只能查看数据类型的描述统计信息，对于其他类型的数据不显示，比如字符串类型姓名（name），客舱号（Cabin）
这很好理解，因为描述统计指标是计算数值，所以需要该列的数据类型是数据。
Pandas Series.mean()函数返回给定Series对象中基础数据的平均值。
'''
# 获取数据类型列的描述统计信息
full.describe()

In [ ]:
full.info()

我们发现数据一共有1309行,其中Survided只有891行,说明有418行数据没有标签,也就是我们要预测的数据
Age(年龄)有1046行数据,说明有1309-1046=263行数据没有年龄,也就是说有263个乘客的年龄没有记录
Fare(船票价格)有1308行数据,只有一个乘客没有记录
Embarked(登船港口)有1307行数据,有两个乘客没有登船港口记录
Cabin(客舱号)只有295行数据,有1007个乘客没有客舱号记录


In [ ]:
def PrintMissing(your_dataframe): 
    """
    打印缺失数据统计信息
    :param 表示数据集
    :return: 返回一个新的dataframe，包含三列：'缺失数据'、'缺失率'、'数据类型'
    """
    drop_sum = your_dataframe.isnull().sum() # 统计每列缺失值个数
    drop_sum_not_zero = drop_sum[drop_sum > 0] # 只保留有缺失值的列
    df1 = pd.DataFrame({'缺失数据': drop_sum_not_zero}) # 创建一个新的dataframe，列名为'缺失数据'
    df2 = pd.DataFrame({'缺失率': drop_sum_not_zero/your_dataframe.shape[0]}) # 创建一个新的dataframe，列名为'缺失率'
    keys = drop_sum_not_zero.to_dict() # 将drop_sum_not_zero转换为字典
    df_dtypes = pd.DataFrame({'数据类型': {key: your_dataframe[key].dtype for key in keys}}) # 创建一个新的dataframe，列名为'数据类型'
    new_df = pd.concat([df1, df2.round(4), df_dtypes], axis=1)
    return new_df

print()
print('缺失数据统计如下（总数 %d）：' % full.shape[0])
PrintMissing(full)

# 数据清洗
很多机器学习算法为了训练模型，要求所传入的特征中不能有空值。

- 如果是数值类型，用平均值取代
- 如果是分类数据，用最常见的类别取代
- 使用模型预测缺失值，例如：K-NN